# Document AI General Document Splitter

This notebook demonstrates how to use Document Splitter to parse a simple PDF document with multiple scanned files to separate documents on page logical boundaries.

In [ ]:
# Install necessary Python libraries and restart your kernel after.
!python -m pip install -r ../requirements.txt

## Set your Processor Variables

In [ ]:
# TODO(developer): Fill these variables with your values before running the sample
PROJECT_ID= 'YOUR_GCP_PROJECT_ID'
LOCATION = 'eu' # Format is 'us' or 'eu'
PROCESSOR_ID = 'YOUR_DOCAI_PROCESSOR_ID' # Create processor in Cloud Console
FILE_PATH = '../resources/general/multi-document.pdf'

Now let's define the function to process the document with Document AI python client 

In [ ]:
# Import necessary Python modules
from google.cloud import documentai_v1beta3 as documentai

In [64]:
def process_document_sample(
    project_id: str, location: str, processor_id: str, file_path: str
):

    # Instantiates a client
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}
    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    with open(file_path, "rb") as image:
        image_content = image.read()

    # Read the file into memory
    document = {"content": image_content, "mime_type": "application/pdf"}

    # Configure the process request
    request = {"name": name, "document": document}

    # Recognizes text entities in the PDF document
    result = client.process_document(request=request)

    document = result.document

    print("Document processing complete.")

    # For a full list of Document object attributes, please reference this page: https://googleapis.dev/python/documentai/latest/_modules/google/cloud/documentai_v1beta3/types/document.html#Document

    document_pages = document.pages

    # Read the text recognition output from the processor
    text = document.text
    print("The document contains the following text (first 100 charactes):")
    print(text[:100])
    
    # Read the detected page split from the processor
    print("\nThe processor detected the following page split entities:")
    print_pages_split(text, document)


def print_pages_split(text: str, document: dict):
    """
    Document AI identifies possible page splits
    in document. This function converts page splits
    to text snippets and prints it.    
    """
    for i, entity in enumerate(document.entities):
        confidence = entity.confidence
        text_entity = ''
        for segment in entity.text_anchor.text_segments:
            start = segment.start_index
            end = segment.end_index
            text_entity += text[start:end]
        pages = [p.page for p in entity.page_anchor.page_refs]
        print(f"*** Entity number: {i}, Split Confidence: {confidence} ***")
        print(f"*** Pages numbers: {[p for p in pages]} ***\nText snippet: {text_entity[:100]}")

We can now run the processor on the sample multi-document pdf.

In [ ]:
process_document_sample(PROJECT_ID, LOCATION, PROCESSOR_ID, FILE_PATH)

You should see the output similar to:
```
Document processing complete.
The document contains the following text (first 100 charactes):
FakeDoc M.D.
HEALTH INTAKE FORM
Please fill out the questionnaire carefully. The information you pro

The processor detected the following page split entities:
*** Entity number: 0, Split Confidence: 0.21864357590675354 ***
*** Pages numbers: [0, 1] ***
Text snippet: FakeDoc M.D.
HEALTH INTAKE FORM
Please fill out the questionnaire carefully. The information you pro
*** Entity number: 1, Split Confidence: 0.970017671585083 ***
*** Pages numbers: [2] ***
Text snippet: Invoice
DATE: 01/01/1970
INVOICE: NO. 001
FROM: Company ABC
user@companyabc.com
TO: John Doe
johndoe
```

---------
Congratulations, you've successfully used the Document AI API to extract page logical boundaries from a multipage document. We encourage you to experiment with other documents.